In [1]:
import folium
import numpy as np
import pandas as pd

In [2]:
df = pd.read_csv('housing_with_zips.csv')
df

,url,address,neighborhood,rent,beds,baths,flexible_rooms,zip
0,https://www.renthop.com/listings/10-montieth-s...,"10 Montieth Street, Apt 341","Bushwick, Northern Brooklyn, Brooklyn",1908,0,1.0,0,11206.0
1,https://www.renthop.com/listings/225-east-10th...,"225 East 10th Street, Apt 5BB","East Village, Downtown Manhattan, Manhattan",1800,0,1.0,0,10003.0
2,https://www.renthop.com/listings/225-east-10th...,"225 East 10th Street, Apt 3EE","East Village, Downtown Manhattan, Manhattan",1800,0,1.0,0,10003.0
3,https://www.renthop.com/listings/25-hillside-a...,"25 Hillside Avenue, Apt 2B","Fort George, Washington Heights, Upper Manhatt...",1776,1,1.0,0,10040.0
4,https://www.renthop.com/listings/525-west-52nd...,"525 West 52nd Street, Apt 2NN","Hell's Kitchen, Midtown Manhattan, Manhattan",5600,2,2.0,0,10019.0
...,...,...,...,...,...,...,...,...
9224,https://www.renthop.com/listings/east-48th-str...,East 48th Street,"Turtle Bay, Midtown East, Midtown Manhattan, M...",4958,3,2.0,0,NaN
9225,https://www.renthop.com/listings/west-31st-str...,West 31st Street,"Chelsea, Midtown Manhattan, Manhattan",3735,1,1.0,0,NaN
9226,https://www.renthop.com/listings/1093-dean-st/...,"1093 Dean Street, Apt 7","Crown Heights, Central Brooklyn, Brooklyn",1760,0,1.0,0,11216.0
9227,https://www.renthop.com/listings/wall-st/309/1...,Wall St,"Financial District, Downtown Manhattan, Manhattan",3450,1,1.0,1,NaN


In [3]:
df = df[df.zip.notnull()].reset_index(drop=True)
df

,url,address,neighborhood,rent,beds,baths,flexible_rooms,zip
0,https://www.renthop.com/listings/10-montieth-s...,"10 Montieth Street, Apt 341","Bushwick, Northern Brooklyn, Brooklyn",1908,0,1.0,0,11206.0
1,https://www.renthop.com/listings/225-east-10th...,"225 East 10th Street, Apt 5BB","East Village, Downtown Manhattan, Manhattan",1800,0,1.0,0,10003.0
2,https://www.renthop.com/listings/225-east-10th...,"225 East 10th Street, Apt 3EE","East Village, Downtown Manhattan, Manhattan",1800,0,1.0,0,10003.0
3,https://www.renthop.com/listings/25-hillside-a...,"25 Hillside Avenue, Apt 2B","Fort George, Washington Heights, Upper Manhatt...",1776,1,1.0,0,10040.0
4,https://www.renthop.com/listings/525-west-52nd...,"525 West 52nd Street, Apt 2NN","Hell's Kitchen, Midtown Manhattan, Manhattan",5600,2,2.0,0,10019.0
...,...,...,...,...,...,...,...,...
4649,https://www.renthop.com/listings/36-west-26th-...,36 West 26th Street,"NoMad, Midtown Manhattan, Manhattan",1450,0,2.0,0,10010.0
4650,https://www.renthop.com/listings/1150-presiden...,"1153 President Street, Apt 4E","Crown Heights, Central Brooklyn, Brooklyn",2200,2,1.0,0,11225.0
4651,https://www.renthop.com/listings/269-e-52nd-st...,269 E 52nd Street,"East Flatbush, Central Brooklyn, Brooklyn",2450,3,1.0,0,11203.0
4652,https://www.renthop.com/listings/28-powers-str...,"28 Powers Street, Apt 2L","East Williamsburg, Williamsburg, Northern Broo...",2825,2,1.5,0,11211.0


## Visualizing the data

In [15]:
zdf_mean = df.groupby('zip')['rent'].mean().to_frame('avg_rent')\
.sort_values(by='avg_rent', ascending=False).reset_index()
zdf_mean

,zip,avg_rent
0,10282.0,6249.166667
1,10007.0,5875.111111
2,10044.0,4933.000000
3,12440.0,4800.000000
4,10069.0,4428.888889
...,...,...
111,11219.0,1600.000000
112,10454.0,1600.000000
113,11214.0,1597.500000
114,7735.0,1557.000000
